##  Trocadilho Generator 

Notebook modified from the original by [Max Woolf](http://minimaxir.com)

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read this [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use the original notebook.

In [2]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import requests
from datetime import datetime
from google.colab import files

## GPU

Verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Sun Nov  7 21:02:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 471Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 966kit/s]
Fetching hparams.json: 1.05Mit [00:00, 573Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:11, 6.97Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 519Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.18Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.15Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Downloading a Text File to be Trained

In [5]:
url = "https://raw.githubusercontent.com/lucasbarross/trocadilho-generator/main/training/jokes.txt"
data = requests.get(url)
file_name = "jokes.txt"

with open(file_name, 'w') as f:
  f.write(data.text)

gpt2.copy_file_from_gdrive("../../"+file_name)

## Training the Model

The next cell will start the actual finetuning/training of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              learning_rate=1e-5,
              model_name='124M',
              steps=1500,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


dataset has 253358 tokens
Training...
[10 | 45.54] loss=4.12 avg=4.12
[20 | 87.10] loss=3.70 avg=3.91
[30 | 128.70] loss=3.81 avg=3.88
[40 | 170.27] loss=3.94 avg=3.89
[50 | 211.83] loss=3.89 avg=3.89
[60 | 253.36] loss=3.57 avg=3.84
[70 | 294.73] loss=3.59 avg=3.80
[80 | 336.49] loss=3.49 avg=3.76
[90 | 378.20] loss=3.72 avg=3.75
[100 | 420.13] loss=3.47 avg=3.73
[110 | 462.18] loss=3.50 avg=3.70
[120 | 504.25] loss=3.38 avg=3.67
[130 | 546.32] loss=3.44 avg=3.66
[140 | 588.28] loss=2.97 avg=3.60
[150 | 630.29] loss=3.35 avg=3.59
[160 | 672.28] loss=3.24 avg=3.56
[170 | 714.28] loss=3.37 avg=3.55
[180 | 756.24] loss=3.31 avg=3.54
[190 | 798.18] loss=3.29 avg=3.52
[200 | 840.11] loss=3.23 avg=3.51
======== SAMPLE 1 ========
s are better because we eat and do our job, better because we have more time. And so for me it was as if people don't care at all about, and not just for, a football player's time. So when he got his contract, it was just for him. It's a bit strange, because it make

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [7]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [3]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1500
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1500


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [5]:
gpt2.generate(sess, run_name='run1')

Aremen, November 3 (CNA) - A Colombian man was sentenced to 10-20 years in prison for evading an EU arrest warrant.

Carlos Boulos, 40, of Amedeiros, was sent to a Maracay for 5.5 years. Mina Manqueirão, a social worker at Amedeiros hospital, said Boulos' crime was "no crime at all".

"No crime at all. No crime, no crime at all. I am a social worker, not a crime, no crime at all," Manqueirão said.

READ MORE:

* Evacueil de Água n'ava atração de fazer em ônibus

* Evacueil de Água n'ava atração de só naquele líderada? Só computadorá, perguntaram de ação de contra-infóbienlandos e com todo mundo e manzar) (Por que uma biblioteca dá um fusca trontário)

At a time when most crime in the continent is directed at the top, it is important to point out.

"No crime at all. No crime, no crime at all. I am a social worker, not a crime." -Carlos A. Boulos

A remit of the interior and social protection and the interior justice so far is to decide on a term for Boulos' crime, and decide on a term f

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [47]:
gpt2.generate(sess,
              length=140,
              temperature=0.8,
              top_k=40,
              prefix='Quando',
              truncate='<|endofjoke|>',
              nsamples=10,
              batch_size=1,
              )

Quando a pessoa atravessar com o pia ele! 
Quando para lá, uma coisa. 
Quando estava que ele se chama um dia. 
Quando estou e aparecida? A com falta 
Quando a dor em casa? O Corrinho 
Quando estava a meijo? Porque ela estava a lavender. 
Quando quisquando estava no meio do natal? Uma filha e ainda no tributa do céu, a comprar o nome. 
Quando, no churrasco no bar? O churrasco no céu! 
Quando é espirram? Na frita. 
Quando! 


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.